In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
import os
os.chdir('/content/gdrive/My Drive/AGGCN-master/semeval/dataset/semeval')

In [0]:
import json
with open('test.json', 'r') as f:
    distros_dict = json.load(f)
# sentence_sub_obj = dict()
# for distro in distros_dict:
#     sentence_sub_obj[" ".join(distro['token'])] = {k:distro[k] for k in ['subj_start', 'subj_end', 'obj_start', 'obj_end']}

In [0]:
import stanza
stanza.download('en') # download English model
nlp = stanza.Pipeline('en') # initialize English neural pipeline

2020-05-22 18:12:28 INFO: Downloading default packages for language: en (English)...
2020-05-22 18:13:06 INFO: Finished downloading models and saved to /root/stanza_resources.
2020-05-22 18:13:06 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | ewt       |
| pos       | ewt       |
| lemma     | ewt       |
| depparse  | ewt       |
| ner       | ontonotes |

2020-05-22 18:13:06 INFO: Use device: cpu
2020-05-22 18:13:06 INFO: Loading: tokenize
2020-05-22 18:13:06 INFO: Loading: pos
2020-05-22 18:13:07 INFO: Loading: lemma
2020-05-22 18:13:07 INFO: Loading: depparse
2020-05-22 18:13:08 INFO: Loading: ner
2020-05-22 18:13:09 INFO: Done loading processors!


In [0]:
# for index, value in zip(range(len(sentences)), sentences):
#   print(value)allmo_sentences
from tqdm import tqdm
all_sentences = dict()
for i,val in enumerate(tqdm(sentence_sub_obj)):
  doc = nlp(str(val))
  all_sentences[val] = [(word.text, word.xpos, word.head, word.deprel) for sent in doc.sentences for word in sent.words]

  0%|          | 22/7975 [00:08<1:00:02,  2.21it/s]

KeyboardInterrupt: ignored

In [20]:
from tqdm import tqdm

dataset_NER = []
for distro in tqdm(distros_dict):
    token_wise = []
    start_subj = distro['subj_start']
    end_subj = distro['subj_end']
    start_obj = distro['obj_start']
    end_obj = distro['obj_end']
    current_subj = False
    current_obj = False
    for i,(tok, pos) in enumerate(zip(distro['token'], distro['stanford_pos'])):
      if current_subj:
        bio_tag = 'I-SUBJ'
      elif current_obj:
        bio_tag = 'I-OBJ'
      else:
        bio_tag = 'O'

      if i == start_subj:
        bio_tag = 'B-SUBJ'
        current_subj = True
      if i == end_subj:
        current_subj = False

      if i == start_obj:
        bio_tag = 'B-OBJ'
        current_obj = True
      if i == end_obj:
        current_obj = False
      if i == 0:
        token_wise.append((distro['id'], tok, pos, bio_tag))
      else:
        token_wise.append((None, tok, pos, bio_tag))
    dataset_NER.append(token_wise)



100%|██████████| 2717/2717 [00:00<00:00, 61252.27it/s]


In [0]:
import pandas as pd
pd.DataFrame(sum(dataset_NER, []), columns=['Sentence #', 'tok', 'pos', 'tag']).to_csv('ner_test.csv')

In [0]:
k = []
for distro in distros_dict:
    k.append(distro['stanford_pos'])

In [0]:
dataset_NER[0]

[('The', 'DT', 'O'),
 ('system', 'NN', 'O'),
 ('as', 'IN', 'O'),
 ('described', 'VBD', 'O'),
 ('above', 'RB', 'O'),
 ('has', 'VBZ', 'O'),
 ('its', 'PRP$', 'O'),
 ('greatest', 'JJS', 'O'),
 ('application', 'NN', 'O'),
 ('in', 'IN', 'O'),
 ('an', 'DT', 'O'),
 ('arrayed', 'JJ', 'O'),
 ('configuration', 'NN', 'B-SUBJ'),
 ('of', 'IN', 'O'),
 ('antenna', 'NN', 'O'),
 ('elements', 'NNS', 'B-OBJ'),
 ('.', '.', 'O')]